In [4]:
import pandas as pd
RES_scopus = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Scopus files/RES_scopus.xlsx')
print(RES_scopus.shape)
#RES_scopus = RES_scopus.loc[RES_scopus['year'] >= 1980]
RES_scopus = RES_scopus.sort_values(by=['year'])
RES_scopus = RES_scopus.reset_index(drop=True)
print(RES_scopus.shape)


(2832, 14)
(2832, 14)


In [2]:
# Creating a dictionary for consolidation
# Read in from Excel sheet, 2 columns first one with incorrect spelling and second one with correct spellig
import pandas as pd

cities = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Cities')
#cities = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Cities')
cities = cities[['Incorrect spelling', 'Correct spelling']]

misspellings = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Misspellings')
#misspellings = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Misspellings')
misspellings = misspellings[['Incorrect spelling', 'Correct spelling']]

comma_in_name = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Comma in name')
#comma_in_name = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Comma in name')
comma_in_name = comma_in_name[['Incorrect spelling', 'Correct spelling']]

colleges_full = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Colleges Full')
#colleges_full = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Colleges Full')
colleges_full = colleges_full[['Incorrect spelling', 'Correct spelling']]

colleges = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Colleges')
#colleges = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Colleges')

correct = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Correct')
#correct = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Correct')

scopus_removing = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Scopus')

#Remove cities
frames = [misspellings, comma_in_name,colleges_full ,colleges,correct, scopus_removing]
consolidated_affiliations = pd.concat(frames)
print(consolidated_affiliations.shape)

for i in range(len(consolidated_affiliations)):
    #Replaceing title case and stripping leading space
    #affs.iat[i,0] = affs.iat[i,0].strip().title()
    consolidated_affiliations.iloc[i]['Incorrect spelling'] = consolidated_affiliations.iloc[i]['Incorrect spelling'].strip().title()
    #affs.iat[i,0] = affs.iat[i,0].strip().upper()

consolidated_affiliations_dict = consolidated_affiliations.set_index('Incorrect spelling').to_dict()['Correct spelling']
print(consolidated_affiliations_dict)

(1656, 2)
{'Bell Laboratories, Inc': 'Bell Laboratories', 'Board Of Governors Of The Federal Reseree System': 'Board Of Governors Of The Federal Reserve System', 'Board Of Governors Of The Federal Reseroe System': 'Board Of Governors Of The Federal Reserve System', 'Board Of Governors, Federal Reserve System': 'Board Of Governors Of The Federal Reserve System', 'Boston Unxversity': 'Boston University', 'Broum University': 'Brown University', 'California State University, Long Beach': 'California State University At Long Beach', 'California State University, Northridge': 'California State University At Northridge', 'California State Unizersity, Fullerton': 'California State University At Fullerton', 'Cambridge University': 'University of Cambridge', 'Carnegie Mellon University': 'Carnegie-Mellon University', 'Carnegie- Mellon University': 'Carnegie-Mellon University', 'Carnegie- Mellon Unizersity': 'Carnegie-Mellon University', 'Cornell Unisersity': 'Cornell University', 'Department Of 

In [7]:
import re
from rapidfuzz import fuzz
import numpy as np
#for i in range(1577,1578):
RES_scopus = RES_scopus.replace(np.nan, '', regex=True)

RES_scopus["0_author_affs_consolidated"] = ""
RES_scopus["1_author_affs_consolidated"] = ""
RES_scopus["2_author_affs_consolidated"] = ""
RES_scopus["3_author_affs_consolidated"] = ""
RES_scopus["4_author_affs_consolidated"] = ""
RES_scopus["5_author_affs_consolidated"] = ""
RES_scopus["0_author_affs_consolidated_0"] = ""
RES_scopus["0_author_affs_consolidated_1"] = ""
RES_scopus["0_author_affs_consolidated_2"] = ""
RES_scopus["1_author_affs_consolidated_1"] = ""
RES_scopus["1_author_affs_consolidated_2"] = ""
RES_scopus["2_author_affs_consolidated_1"] = ""
RES_scopus["2_author_affs_consolidated_2"] = ""
RES_scopus["3_author_affs_consolidated_0"] = ""
RES_scopus["3_author_affs_consolidated_1"] = ""
RES_scopus["3_author_affs_consolidated_2"] = ""
RES_scopus["1_author_affs"] = ""
RES_scopus["2_author_affs"] = ""
RES_scopus["3_author_affs"] = ""
RES_scopus["4_author_affs"] = ""
RES_scopus["5_author_affs"] = ""

association_list = ['a--','b--','c--','d--','e--','f--','g--','h--']
#for i in range(1577,1578):
#for i in range(1594,1595):
#for i in range(790,800):
for i in range(len(RES_scopus)):
    print(i)

    #print(RES_scopus.loc[i,'authors'])
    #print(RES_scopus.loc[i,'affiliations'])

# Checking for if the authors and affiliations are split by a-- b-- c--
    ratio = fuzz.partial_ratio(RES_scopus.loc[i,'authors'] , 'a--')
    if ratio < 95:
        RES_scopus.loc[i,'0_author'] = RES_scopus.iloc[i]['authors']
        RES_scopus.loc[i,'0_author_affs_0'] = RES_scopus.iloc[i]['affiliations'].title().strip()
        #print(RES_scopus.loc[i,'0_author_affs_0'])
        list = []
        for key in consolidated_affiliations_dict.keys():
            if key in RES_scopus.iloc[i]['0_author_affs_0']:
                print(key)
                list.append(key)
                print(consolidated_affiliations_dict[key])
                print(len(list))
                affil_auth = re.sub(key, consolidated_affiliations_dict[key],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',')
                #print(affil_auth)
                RES_scopus.loc[i,'0_author_affs_consolidated_0'] = re.sub(r'\.','',affil_auth).title().strip()
                #print(RES_scopus.loc[i,'0_author_affs_consolidated_0'])
                
#If there is more than 1 match in the consolidation it needs to match on the longer one, solves for college issue
                if len(list) <= 1:
                    RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(key, consolidated_affiliations_dict[key],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',')
                else:
                    print(consolidated_affiliations_dict[list[0]])
                    print(consolidated_affiliations_dict[list[1]])
                    # For colleges
                    if len(list) ==2:
                        if(len(consolidated_affiliations_dict[list[0]]) == len(consolidated_affiliations_dict[list[1]])):
                            #print("HERE")
                            if len(list[0]) > len(list[1]):
                                print("If")
                                RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(list[0], consolidated_affiliations_dict[list[0]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',')
                                #print(list[0])
                            # Full  University Of Illinois System Urbana-Champaign issue
                            elif len(list[1]) > len(list[0]):
                                print("Elif")
                                RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(list[1], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',')
                            else:
                                print("else")
                                first_fix = re.sub(list[1], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',') 
                                RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                        else:
                            first_fix = re.sub(list[0], consolidated_affiliations_dict[list[0]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',') 
                            #print(first_fix)
                            RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
# ['University Of California At Berkeley', 'California At Berkeley', 'Department Of Economics, University Of California At Berkeley', 'University Of California']
                    else:
                        print('long len')
                        if(len(consolidated_affiliations_dict[list[0]]) == len(consolidated_affiliations_dict[list[1]])== len(consolidated_affiliations_dict[list[2]])):
                            if(len(list[0]) > len(list[1]) and len(list[0]) > len(list[2])):
                                print("If")
                                RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(list[0], consolidated_affiliations_dict[list[0]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',')
                                #print(list[0])
                            # Full  University Of Illinois System Urbana-Champaign issue
                            elif(len(list[1]) > len(list[0]) and len(list[1]) > len(list[2])):
                                print("Elif")
                                RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(list[1], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',')
                            elif(len(list[2]) > len(list[0]) and len(list[2]) > len(list[1])):
                                print("Elif")
                                RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(list[2], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',')
                            else:
                                print("else")
                                first_fix = re.sub(list[1], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',') 
                                RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                        else:
                            first_fix = re.sub(list[0], consolidated_affiliations_dict[list[0]],RES_scopus.iloc[i]['0_author_affs_0']).strip().rstrip(',') 
                            #print(first_fix)
                            RES_scopus.loc[i,'0_author_affs_consolidated_0']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
        RES_scopus.loc[i,'0_author_affs_consolidated_0'] = re.sub(r'\.','',RES_scopus.iloc[i]['0_author_affs_consolidated_0']).title().strip()
        print(RES_scopus.loc[i,'0_author_affs_consolidated_0'])
        print("False")
    else:

        # Split on new lines
        authors = RES_scopus.iloc[i]['authors']
        authors_by_line = re.split(r"\n" , authors)
            
        for auth_no in range(len(authors_by_line)):
            RES_scopus.loc[i,str(auth_no)+'_author'] = authors_by_line[auth_no]
            print(RES_scopus.loc[i,str(auth_no)+'_author'])
            count = 0

        # For a-- b-- c--

            for association in association_list:
                # Checking for a-- or b-- in the authors line
                ratio = fuzz.partial_ratio(RES_scopus.loc[i,str(auth_no)+'_author'] , association)
                if ratio > 95:
                    print("True")

                    #Splitting the affiliaitons by line
                    affiliations = RES_scopus.iloc[i]['affiliations']
                    affiliations_by_line = re.split(r"\n" , affiliations)

                    # Looping through the affiliations by line checking for associated author a-- b--
                    for aff_no in range(len(affiliations_by_line)):
                    #Checking for a-- or b--
                        ratio_aff = fuzz.partial_ratio(affiliations_by_line[aff_no], association)
                        if ratio_aff > 95:
                            affil_auth = re.sub(r'a--|b--|c--|d--|e--|f--|g--|h--','',affiliations_by_line[aff_no]).title().strip()
                            print(affil_auth)
                            RES_scopus.loc[i,str(auth_no)+'_author_affs_'+str(count)] = affil_auth

                            for key in consolidated_affiliations_dict.keys():
                                if key in RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]:
                                    #print(key)
                                    #print(consolidated_affiliations_dict[key])
                                    affil_auth = re.sub(key, consolidated_affiliations_dict[key],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',')
                                    #print(affil_auth)
                                    RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)] = re.sub(r'\.','',affil_auth).title().strip()
                                    #print(RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)])

                                    #RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)] = re.sub(r'\.','',affil_auth).title().strip()
                                    #print(RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)])

                    #If there is more than 1 match in the consolidation it needs to match on the longer one, solves for college issue
                                    if len(list) <= 1:
                                        RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(key, consolidated_affiliations_dict[key],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',')
                                    else:
                                        print(consolidated_affiliations_dict[list[0]])
                                        print(consolidated_affiliations_dict[list[1]])
                                        # For colleges
                                        if len(list) ==2:
                                            if(len(consolidated_affiliations_dict[list[0]]) == len(consolidated_affiliations_dict[list[1]])):
                                                #print("HERE")
                                                if len(list[0]) > len(list[1]):
                                                    print("If")
                                                    RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(list[0], consolidated_affiliations_dict[list[0]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',')
                                                    #print(list[0])
                                                # Full  University Of Illinois System Urbana-Champaign issue
                                                elif len(list[1]) > len(list[0]):
                                                    print("Elif")
                                                    RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(list[1], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',')
                                                else:
                                                    print("else")
                                                    first_fix = re.sub(list[1], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',') 
                                                    RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                                            else:
                                                first_fix = re.sub(list[0], consolidated_affiliations_dict[list[0]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',') 
                                                #print(first_fix)
                                                RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                    # ['University Of California At Berkeley', 'California At Berkeley', 'Department Of Economics, University Of California At Berkeley', 'University Of California']
                                        else:
                                            print('long len')
                                            if(len(consolidated_affiliations_dict[list[0]]) == len(consolidated_affiliations_dict[list[1]])== len(consolidated_affiliations_dict[list[2]])):
                                                if(len(list[0]) > len(list[1]) and len(list[0]) > len(list[2])):
                                                    print("If")
                                                    RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(list[0], consolidated_affiliations_dict[list[0]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',')
                                                    #print(list[0])
                                                # Full  University Of Illinois System Urbana-Champaign issue
                                                elif(len(list[1]) > len(list[0]) and len(list[1]) > len(list[2])):
                                                    print("Elif")
                                                    RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(list[1], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',')
                                                elif(len(list[2]) > len(list[0]) and len(list[2]) > len(list[1])):
                                                    print("Elif")
                                                    RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(list[2], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',')
                                                else:
                                                    print("else")
                                                    first_fix = re.sub(list[1], consolidated_affiliations_dict[list[1]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',') 
                                                    RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                                            else:
                                                first_fix = re.sub(list[0], consolidated_affiliations_dict[list[0]],RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',') 
                                                #print(first_fix)
                                                RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)]  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                            RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)] = re.sub(r'\.','',RES_scopus.iloc[i][str(auth_no)+'_author_affs_consolidated_'+str(count)]).title().strip()
                            print(RES_scopus.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(count)])
                            #consolidation = re.sub(r', United States|, Israel|, United Kingdom|','',affiliations_by_line[aff_no]).title().strip()
                            #print(consolidation)
                            #RES_scopus.loc[i,str(auth_no)+'_author_affs_Consolidated_'+str(count)] = consolidation
                            #for key in consolidated_affiliations_dict.keys():
                            #    if key in RES_scopus.iloc[i][str(auth_no)+'_author_affs_'+str(count)]:
                            #        RES_scopus.loc[i,'0_author_affs_consolidated_0'+str(count)]  = re.sub(key, consolidated_affiliations_dict[key],RES_scopus.loc[i,str(auth_no)+'_author_affs_'+str(count)]).strip().rstrip(',')
                            print("True 95")         
                            count = count +1
                            print('count ' , count)

RES_scopus.iloc[[409]].head()


0
, United Kingdom
.
1
London
False
1

False
2
, United States
.
1
Cambridge
False
3
, United States
.
1
Chicago
False
4
, United Kingdom
.
1
Birmingham
False
5
, United States
.
1
Columbus, Oh
False
6
, United Kingdom
.
1
London
False
7

False
8

False
9

False
10

False
11
, United Kingdom
.
1
London
False
12

False
13

False
14
, United Kingdom
.
1
Manchester
False
15
, United Kingdom
.
1
London
False
16

False
17

False
18
, United Kingdom
.
1
Dundee
False
19
, United States
.
1
Berkeley, Ca, United States
Berkeley (City)
2
.
Berkeley (City)
Berkeley, Ca
False
20

False
21

False
22
, United Kingdom
.
1
London
False
23

False
24

False
25
, United States
.
1
Berkeley, Ca, United States
Berkeley (City)
2
.
Berkeley (City)
Berkeley, Ca
False
26

False
27
, United Kingdom
.
1
London
False
28
, United Kingdom
.
1
Cambridge
False
29
, United Kingdom
.
1
Edinburgh
False
30
, United States
.
1
Chicago
False
31

False
32

False
33
, United Kingdom
.
1
Oxford
False
34
, United Kingdom
.
1
C

,authors,title,journal,volume,issue,year,pages,DOI,affiliations,abstract,...,0_author_affs_consolidated_3,1_author_affs_3,1_author_affs_consolidated_3,2_author_affs_3,2_author_affs_consolidated_3,3_author_affs_3,3_author_affs_consolidated_3,4_author,4_author_affs_0,4_author_affs_consolidated_0
409,"Lewis, J.P.","Aggregation, peaks and troughs",Review of Economic Studies,29.0,1.0,1961,79-84,10.2307/2296185,"Manchester, United Kingdom",,...,,,,,,,,,,


In [8]:
print(RES_scopus.columns.values)

['authors' 'title' 'journal' 'volume' 'issue' 'year' 'pages' 'DOI'
 'affiliations' 'abstract' 'citations' 'document type' 'index keywords'
 'author keywords' '0_author_affs_consolidated'
 '1_author_affs_consolidated' '2_author_affs_consolidated'
 '3_author_affs_consolidated' '4_author_affs_consolidated'
 '5_author_affs_consolidated' '0_author_affs_consolidated_0'
 '0_author_affs_consolidated_1' '0_author_affs_consolidated_2'
 '1_author_affs_consolidated_1' '1_author_affs_consolidated_2'
 '2_author_affs_consolidated_1' '2_author_affs_consolidated_2'
 '3_author_affs_consolidated_0' '3_author_affs_consolidated_1'
 '3_author_affs_consolidated_2' '1_author_affs' '2_author_affs'
 '3_author_affs' '4_author_affs' '5_author_affs' '0_author'
 '0_author_affs_0' '0_author_affs_1' '0_author_affs_2' '1_author'
 '1_author_affs_0' '1_author_affs_consolidated_0' '1_author_affs_1'
 '1_author_affs_2' '2_author' '2_author_affs_0'
 '2_author_affs_consolidated_0' '2_author_affs_1' '2_author_affs_2'
 '3_auth

In [10]:
RES_scopus = RES_scopus[['authors' ,'title', 'journal', 'volume', 'issue',
'year', 'pages' , 'DOI',
'affiliations' ,
'document type' ,
 '0_author',
 '0_author_affs_0', 
 '0_author_affs_consolidated_0',
 '0_author_affs_1', 
 '0_author_affs_consolidated_1', 
 '0_author_affs_2', 
 '0_author_affs_consolidated_2',

 '1_author',
 '1_author_affs_0', 
 '1_author_affs_consolidated_0',
 '1_author_affs_1', 
 '1_author_affs_consolidated_1',

 '2_author',
 '2_author_affs_0',
 '2_author_affs_consolidated_0',
 '2_author_affs_1',
 '2_author_affs_consolidated_1',

 '3_author',
 '3_author_affs_0', 
 '3_author_affs_consolidated_0',

  '4_author',
 '4_author_affs_0', 
 '4_author_affs_consolidated_0',]]

In [ ]:
# Excel output to get an overview of the data
RES_scopus.head
RES_scopus.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/3_consolidation/RES/Scopus/RES_scopus split consolidated 1940 2010.xlsx') 